In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import time
from scipy.fftpack import dct, dctn
import heapq

In [2]:
class HuffmanTree:
  class Node:
    def __init__(self, value, freq, leftNode=None, rightNode=None, code='', step=0):
      self.value = value
      self.freq = freq
      self.leftNode = leftNode
      self.rightNode = rightNode
      self.code = code
      self.create_step = step
    
    def is_leaf(self):
      return self.value is not None

    def __eq__(self, other):
      stup = self.value, self.freq, self.leftNode, self.rightNode
      otup = other.value, other.freq, other.leftNode, other.rightNode
      return stup == otup

    def __nq__(self, other):
      return not (self == other)

    def __lt__(self, other):
      return self.freq < other.freq

    def __le__(self, other):
      return self.freq < other.freq or self.freq == other.freq

    def __gt__(self, other):
      return not (self <= other)

    def __ge__(self, other):
      return not (self < other)

  def __init__(self, data=None):
    self.data = data
    self.heap = []
    self.code = ''
  
  def build(self):
    if self.data is None:
      raise ValueError('Data should not be None')
    values, count = np.unique(self.data, return_counts=True)
    count = count / np.sum(count)
    for value, freq in zip(values, count):
      heapq.heappush(self.heap, self.Node(value, freq))
    self.num_code = len(self.heap)

    step = 1
    while len(self.heap) > 1: 
      min1 = heapq.heappop(self.heap)
      min2 = heapq.heappop(self.heap)
      heapq.heappush(self.heap, self.Node(None, min1.freq+min2.freq, min1, min2, step=step))
      step += 1
    self.backward()
    self.mapping_table = {node.value: node.code for node in self.heap}
    return self.mapping_table
  def backward(self):
    while len(self.heap) != self.num_code:
      self.heap.sort(key=lambda x: x.create_step, reverse=True)
      node = self.heap.pop(0)
      
      node1 = node.leftNode
      node2 = node.rightNode
      node1.code = node.code + '0'
      node2.code = node.code + '1'
      self.heap += [node1, node2]
  
  def encode(self):
    if self.code != '':
      return self.code
    for i in self.data:
      self.code += self.mapping_table[i]
    return self.code
  def decode(self, data):
    container = []
    temp = ''
    for i in data:
      temp += i
      if temp in self.mapping_table.values():
        container += [list(self.mapping_table.keys())[list(self.mapping_table.values()).index(temp)]]
        # print('data:\t', temp)
        temp = ''
    print(temp)
    return container



In [ ]:
class JPEG:
  def __init__(self):
    self.quantizer_table = np.array([
        [17, 18, 24, 47, 99, 99, 99, 99],
        [18, 21, 26, 66, 99, 99, 99, 99],
        [24, 26, 56, 99, 99, 99, 99, 99],
        [47, 66, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99]
  ])
    self.zigzag_matrix = np.array([
        [ 0,  1,  5,  6, 14, 15, 27, 28],
        [ 2,  4,  7, 13, 16, 26, 29, 42],
        [ 3,  8, 12, 17, 25, 30, 41, 43],
        [20, 22, 33, 38, 46, 51, 55, 60],
        [21, 34, 37, 47, 50, 56, 59, 61],
        [35, 36, 48, 49, 57, 58, 62, 63],
        [ 9, 11, 18, 24, 31, 40, 44, 53],
        [10, 19, 23, 32, 39, 45, 52, 54]
    ])

  def compress(self, img):
    height, width = img.shape
    if height % 8 + width % 8 != 0:
      raise ValueError('Both width and height must divisible by 3')
    num_block = (height/8) * (width/8)
    indices = np.indices(np.ceil((height/8, width/8)).astype(np.int16)).reshape(2,-1).astype(np.int16) * 8

    dc = np.empty(num_block, 3)
    ac = np.empty(num_block, 63, 3)

    for block_id in range(indices.shape[1]):
      for k in range(3):
        block = img[indices[0, block_id]:indices[0, block_id] + 8, indices[1, block_id]:indices[1, block_id]+8]
        dct_result = dctn(block, norm='ortho')
        quantizer_result = np.int32(dct_result / self.quantizer_table)

        zigzag_ret = [0] * 64
        for i in range(8):
          for j in range(8):
            zigzag_ret[self.zigzag_matrix[i,j]] = quantizer_result[i,j]
        dc[block_id, k] = zigzag_ret[0]
        ac[block_id, :, k] = zigza_ret[:]
    
    huf_dc_y = HuffmanTree(dc[:, 0])
    huf_dc_c = HuffmanTree(dc[:, 1:])
    huf_ac_y = HuffmanTree(ac[])

        
      
    return zigzag_ret

In [ ]:
a = np.random.randint(0, 15, (4,2))
a

array([[14,  0],
       [ 8,  7],
       [ 1, 14],
       [12,  7]])

In [ ]:
a.

array([[14,  8,  1, 12],
       [ 0,  7, 14,  7]])